# ADM-HW4

# 1. Recommendation System with LSH


In [6]:
import os

import kagglehub
import pandas as pd

# sets up pandas table display
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

## 1.1 Data Preparation

In [ ]:

path = kagglehub.dataset_download("grouplens/movielens-20m-dataset")

print("Path to dataset files:", path)

files = os.listdir(path)
files

In [5]:
genome_scores = pd.read_csv("data/genome_scores.csv")
genome_tags = pd.read_csv("data/genome_tags.csv")
link = pd.read_csv("data/link.csv")
movie = pd.read_csv("data/movie.csv")
rating = pd.read_csv("data/rating.csv")
tag = pd.read_csv("data/tag.csv")

# Data Overview

## Content
No demographic information is included. Each user is represented by an `id`, and no other information is provided.

The data are contained in six files:

### 1. `tag.csv`
Contains tags applied to movies by users:
- `userId`
- `movieId`
- `tag`
- `timestamp`

### 2. `rating.csv`
Contains ratings of movies by users:
- `userId`
- `movieId`
- `rating`
- `timestamp`

### 3. `movie.csv`
Contains movie information:
- `movieId`
- `title`
- `genres`

### 4. `link.csv`
Contains identifiers that can be used to link to other sources:
- `movieId`
- `imdbId`
- `tmdbId`

### 5. `genome_scores.csv`
Contains movie-tag relevance data:
- `movieId`
- `tagId`
- `relevance`

### 6. `genome_tags.csv`
Contains tag descriptions:
- `tagId`
- `tag`


## Structure_movie_dataset

![Structure of Movie Dataset](images/structure_movie_dataset.jpg)



In [8]:
tag.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


In [9]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.500,2005-04-02 23:53:47
1,1,29,3.500,2005-04-02 23:31:16
2,1,32,3.500,2005-04-02 23:33:39
3,1,47,3.500,2005-04-02 23:32:07
4,1,50,3.500,2005-04-02 23:29:40


In [11]:
link.head()

,movieId,imdbId,tmdbId
0,1,114709,862.000
1,2,113497,8844.000
2,3,113228,15602.000
3,4,114885,31357.000
4,5,113041,11862.000


In [12]:
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.025
1,1,2,0.025
2,1,3,0.058
3,1,4,0.097
4,1,5,0.147


In [13]:
genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [15]:
print("rating:", rating.columns)
print("links:", link.columns)
print("genome_tags:", genome_tags.columns)
print("genome_scores:", genome_scores.columns)
print("tags:", tag.columns)
print("movies:", movie.columns)

rating: Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
links: Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')
genome_tags: Index(['tagId', 'tag'], dtype='object')
genome_scores: Index(['movieId', 'tagId', 'relevance'], dtype='object')
tags: Index(['userId', 'movieId', 'tag', 'timestamp'], dtype='object')
movies: Index(['movieId', 'title', 'genres'], dtype='object')


## 2. Grouping Movies Together! 
In this section, you will explore clustering algorithms to group the movies you have based on specific features you choose to consider for them.

### 2.1 Feature Engineering 
As you know, the dataset provided isn’t particularly clean or well-structured to represent the features of the movies. Therefore, your first step is to create a more suitable set of attributes (variables, features, covariates) to represent the movies based on the available information. Here are some variables or features you might consider for clustering:

1. ```movieid``` id of each movie 
2. ```genres``` list of genres attached to the movie (given that a movie may have several genres, it’s essential to devise a method to accurately represent the genres for each movie)
3. ```ratings_avg``` the average ratings provided by users for the movie
4. ```relevant_genome_tag``` the most relevant tag to the movie given in the genome set
5. ```common_user_tag``` the most common tag given to the movie by the users 

In addition to the above features, include **at least three additional** features for clustering.

__Note__: If you have accurately identified and applied the methods for representing the features, you should have __more than eight features__! How could this happen? Take a moment to think about it.

# 3. Bonus Question


# 4. Algorithmic Question
